In [1]:
import numpy as np
import pandas as pd

import zipfile
import io
import os
import glob
import time
import re

import folium

# Bergstraße Groundwater data

Hessen provides a fairly reasonable interface for retrieving groundwater data:
https://lgd.hessen.de/mapapps/resources/apps/lgd/index.html?lang=en

The area was manually selected and then, all groundwater station data and measurement series were downloaded.

## 1. Basic cleaning of station data

In [2]:
station_path = './data/groundwater/bergstraße/stationdata_raw/gw_stations_bergstraße_raw.xlsx'
stations = pd.read_excel(station_path)#, sep=';')
stations

,Mst.-ID,Name,Kurzname,Betreiber,Ostwert,Nordwert,Rechtswert,Hochwert,TK-Blatt,Messstellenart,...,Sohltiefe,Filter-Oberkante,Filter-Unterkante,Rohrinnendurchmesser (mm),Grundwasserkörper,Hydrogeol. Teilraum,Stilllegungsdatum,Stilllegungsgrund,GewAnlagen-ID,Bohrarchiv HLNUG
0,12890,BIBLIS (alt),544047.0,Regierungspräsidium Darmstadt,458821.000,5505747.000,3458880.0,5507510.0,6216 - Gernsheim,BR,...,6.25,4.25,6.25,100,2395_3101,03101 - Rheingrabenscholle,"Aug 18, 2010",Ersatz durch Neubau,NaN,225
1,12891,GROSS-ROHRHEIM,544017.0,Regierungspräsidium Darmstadt,462960.000,5505877.000,3463020.0,5507640.0,6216 - Gernsheim,BR,...,3.75,2.75,3.75,35,2395_3101,03101 - Rheingrabenscholle,"Aug 1, 1973",Sonstiges,NaN,11
2,12892,NORDHEIM,544013.0,Regierungspräsidium Darmstadt,455982.000,5505917.000,3456040.0,5507680.0,6216 - Gernsheim,BR,...,7.00,6.00,7.00,35,2395_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,7
3,12893,GROSS-ROHRHEIM,544016.0,Regierungspräsidium Darmstadt,461080.000,5505917.000,3461140.0,5507680.0,6216 - Gernsheim,BR,...,4.90,3.70,4.80,35,2395_3101,03101 - Rheingrabenscholle,"Jun 30, 1982",Sonstiges,NaN,10
4,12894,NORDHEIM (alt),544014.0,Regierungspräsidium Darmstadt,457072.000,5505927.000,3457130.0,5507690.0,6216 - Gernsheim,BR,...,5.20,4.20,5.20,35,2395_3101,03101 - Rheingrabenscholle,"May 1, 2007",Ersatz durch Neubau,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,17129,VIERNHEIM,544273.0,NaN,467596.000,5490910.000,3467658.0,5492667.0,6417 - Mannheim Nordost,BR,...,15.00,3.00,15.00,125,2394_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0
248,17130,LAMPERTHEIM,544274.0,Regierungspräsidium Darmstadt,465688.000,5492192.000,3465750.0,5493950.0,6417 - Mannheim Nordost,BR,...,12.80,8.00,12.00,125,2393_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0
249,17131,VIERNHEIM,544275.0,NaN,469267.000,5491023.000,3469330.0,5492780.0,6417 - Mannheim Nordost,BR,...,11.00,7.00,11.00,125,2394_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0
250,17740,LAMPERTHEIM,NaN,Regierungspräsidium Darmstadt,463546.000,5493941.000,3463607.0,5495700.0,6416 - Mannheim Nordwest,BR,...,15.00,9.00,15.00,125,2393_3101,03101 - Rheingrabenscholle,NaN,NaN,NaN,0


The data needs some basic cleaning before further use.

In [3]:
stations.Stilllegungsdatum.unique()

array(['Aug 18, 2010', 'Aug 1, 1973', nan, 'Jun 30, 1982', 'May 1, 2007',
       'Jun 26, 1973', 'Jan 1, 1970', 'Jan 1, 1973', 'Sep 1, 1976',
       'Nov 9, 1973', 'May 1, 1970', 'May 22, 2000', 'Apr 1, 2004',
       'Oct 30, 1973', 'Jun 25, 1973', 'May 15, 2007', 'Jul 1, 1979',
       'Feb 1, 1967', 'Jan 1, 1976', 'Aug 20, 1973', 'May 1, 1996',
       'Oct 16, 1979', 'Jan 1, 2000', 'May 22, 2007', 'Jan 1, 1966',
       'Jun 1, 1992', 'May 21, 2007', 'Nov 11, 2016', 'May 3, 2011',
       'Dec 21, 2015', 'Aug 23, 2017', 'Oct 1, 1997', 'Jan 1, 1986',
       'Dec 1, 2001', 'Dec 1, 1967', 'May 16, 2007', 'Mar 1, 2010',
       'Nov 14, 2001'], dtype=object)

In [4]:
# drop unnecessary columns
cols_to_keep = ['Mst.-ID', 'Ostwert', 'Nordwert', 
                'Baudatum', 'Stilllegungsdatum', 
                'Geländehöhe', 'Messpunkthöhe']
stations = stations[cols_to_keep]

# assign new column names
new_col_names = {'Mst.-ID': 'station_id', 'Ostwert': 'x', 'Nordwert': 'y',
                 'Baudatum': 'start_date', 'Stilllegungsdatum': 'end_date',
                 'Geländehöhe': 'ground_elev', 'Messpunkthöhe': 'measure_elev'}
stations = stations.rename(columns=new_col_names)
stations = stations.drop_duplicates()

# change date columns to datetime type
stations['start_date'] = pd.to_datetime(stations['start_date'], format='%d.%m.%Y')
stations['end_date'] = pd.to_datetime(stations['end_date'], format='%b %d, %Y')

In [5]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 251
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   station_id    243 non-null    int64         
 1   x             243 non-null    float64       
 2   y             243 non-null    float64       
 3   start_date    243 non-null    datetime64[ns]
 4   end_date      49 non-null     datetime64[ns]
 5   ground_elev   243 non-null    float64       
 6   measure_elev  243 non-null    float64       
dtypes: datetime64[ns](2), float64(4), int64(1)
memory usage: 15.2 KB


In [6]:
stations.head()

,station_id,x,y,start_date,end_date,ground_elev,measure_elev
0,12890,458821.0,5505747.0,1969-01-01,2010-08-18,87.44,87.95
1,12891,462960.0,5505877.0,1951-01-01,1973-08-01,89.96,90.01
2,12892,455982.0,5505917.0,1951-01-01,NaT,89.10,89.40
3,12893,461080.0,5505917.0,1951-01-01,1982-06-30,88.17,88.03
4,12894,457072.0,5505927.0,1951-01-01,2007-05-01,88.13,88.49


In [7]:
stations.to_csv('./data/groundwater/bergstraße/gw_stations_bergstraße.csv', index=False)

## 2. Read groundwater measurement data to df

In [8]:
# Directory containing the folders
folder_path = './data/groundwater/bergstraße/stationdata_raw/'

# File name pattern to search for
file_pattern = 'Wasserstaende*.csv' 

df_list = []

# Check if the item in the directory is a folder
if os.path.isdir(folder_path):
    # Use glob to search for files matching the pattern inside the folder
    files = glob.glob(os.path.join(folder_path, file_pattern))
    
    # Process the found files
    for file in files:
        #print("Found file '{}' in folder '{}'".format(os.path.basename(file), folder))
        df = pd.read_csv(file, sep=';')
        df_list.append(df)


In [9]:
df_list[0].head()

,Mst.-ID,Kurzname,Name,Datum,Abstich (m unter MP),Wasserspiegel (m ü. NN),Wasserspiegel (m unter GOK),Wassertemperatur (°C),Unnamed: 8
0,13595,544240,LAMPERTHEIM,"Jan 1, 1979",9.48,89.76,8.45,0.0,NaN
1,13595,544240,LAMPERTHEIM,"Jan 8, 1979",9.49,89.75,8.46,0.0,NaN
2,13595,544240,LAMPERTHEIM,"Jan 15, 1979",9.89,89.35,8.86,0.0,NaN
3,13595,544240,LAMPERTHEIM,"Jan 29, 1979",9.98,89.26,8.95,0.0,NaN
4,13595,544240,LAMPERTHEIM,"Feb 5, 1979",10.00,89.24,8.97,0.0,NaN


In [10]:
df_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25938 entries, 0 to 25937
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Mst.-ID                      25938 non-null  int64  
 1   Kurzname                     25938 non-null  int64  
 2   Name                         25938 non-null  object 
 3   Datum                        25938 non-null  object 
 4   Abstich (m unter MP)         25938 non-null  float64
 5   Wasserspiegel (m ü. NN)      25938 non-null  float64
 6   Wasserspiegel (m unter GOK)  25938 non-null  float64
 7   Wassertemperatur (°C)        25938 non-null  float64
 8   Unnamed: 8                   0 non-null      float64
dtypes: float64(5), int64(2), object(2)
memory usage: 1.8+ MB


The data needs some basic cleaning before further use.

# continue here!!!

In [11]:
# clean column names
df = pd.concat(df_list)

# drop unnecessary columns
cols_to_keep = ['Mst.-ID', 'Datum', 'Abstich (m unter MP)', 
                'Wasserspiegel (m ü. NN)', 'Wasserspiegel (m unter GOK)',
               'Wassertemperatur (°C)']
df = df[cols_to_keep]

# assign new column names
new_col_names = {'Mst.-ID': 'station_id', 'Datum': 'date', 
                 'Abstich (m unter MP)': 'water_dist', 
                 'Wasserspiegel (m ü. NN)': 'water_level', 
                 'Wasserspiegel (m unter GOK)': 'water_depth',
                'Wassertemperatur (°C)': 'water_temp'}
df = df.rename(columns=new_col_names)

# change date column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')

In [12]:
df.describe()

,station_id,date,water_dist,water_level,water_depth,water_temp
count,578813.000000,578813,578813.000000,578813.000000,578813.000000,578813.000000
mean,13576.277737,1995-01-30 17:50:26.388833536,3.992867,89.109238,3.612984,1.427340
min,12890.000000,1912-10-15 00:00:00,0.000000,0.000000,-0.870000,0.000000
25%,13134.000000,1980-08-11 00:00:00,2.510000,86.920000,2.120000,0.000000
50%,13556.000000,1997-09-22 00:00:00,3.410000,89.380000,3.090000,0.000000
75%,13773.000000,2013-07-15 00:00:00,4.510000,92.230000,4.180000,0.000000
max,17794.000000,2024-03-20 00:00:00,28.160000,102.710000,28.310000,21.300000
std,603.023430,NaN,2.994298,7.304354,2.932078,3.969897


In [13]:
df.head()

,station_id,date,water_dist,water_level,water_depth,water_temp
0,13595,1979-01-01,9.48,89.76,8.45,0.0
1,13595,1979-01-08,9.49,89.75,8.46,0.0
2,13595,1979-01-15,9.89,89.35,8.86,0.0
3,13595,1979-01-29,9.98,89.26,8.95,0.0
4,13595,1979-02-05,10.00,89.24,8.97,0.0


Several rows have erroneous entries (water_level==0, water_temp==0); replace them and related measurements with nan.

In [14]:
# if water_level == 0, entire measurement is corrupted
df.loc[df['water_level'] == 0, ['water_dist', 'water_level', 'water_depth', 'water_temp']] = np.nan

# it water_temp == 0, no temp measurement available
df.loc[df['water_temp'] == 0, ['water_temp']] = np.nan

In [15]:
df.describe()

,station_id,date,water_dist,water_level,water_depth,water_temp
count,578813.000000,578813,575651.000000,575651.000000,575651.000000,66890.00000
mean,13576.277737,1995-01-30 17:50:26.388833536,4.008921,89.598707,3.632829,12.27388
min,12890.000000,1912-10-15 00:00:00,0.000000,81.140000,-0.870000,2.02000
25%,13134.000000,1980-08-11 00:00:00,2.520000,86.940000,2.140000,11.30000
50%,13556.000000,1997-09-22 00:00:00,3.420000,89.420000,3.100000,12.00000
75%,13773.000000,2013-07-15 00:00:00,4.520000,92.250000,4.180000,12.90000
max,17794.000000,2024-03-20 00:00:00,28.160000,102.710000,28.310000,21.30000
std,603.023430,NaN,2.979896,3.129021,2.927834,1.55057


In [16]:
df.to_csv('./data/groundwater/bergstraße/gw.csv', index=False)